<a href="https://colab.research.google.com/github/gillesretiere/langdeck/blob/langdeckAB/langdeck_utils_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata
import ast

!pip3 install pyclean

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

# python .env see -> https://saurabh-kumar.com/python-dotenv/
!pip install python-dotenv
from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()
dotenv_config = dotenv_values(".env")
app_path = dotenv_config["APP_PATH"]

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)


!export PYTHONIOENCODING=utf8

%load_ext autoreload
%autoreload 2
%cd /content/drive/MyDrive/langdeck/
!pyclean .
import sys, os

# .env
from dotenv import load_dotenv
from dotenv import dotenv_values
%cd /content/drive/MyDrive/langdeck/
load_dotenv()
dotenv_config = dotenv_values(".env")
!pip install wget
import wget

#load_dotenv()
#dotenv_config = dotenv_values(".env")

print ("Environnement prêt.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
Found existing installation: gspread-dataframe 3.3.1
Uninstalling gspread-dataframe-3.3.1:
  Successfully uninstalled gspread-dataframe-3.3.1
Mounted at /content/drive
/content/drive/MyDrive/langdeck
/content/drive/MyDrive/langdeck
Cleaning directory .
Total 0 files, 0 directories removed.
/content/drive/MyDrive/langdeck
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=28463c70748dc9a26b82e1c10c507725a0ab725b0736c5aaeb4af8df20cd820d
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Environnement prêt.


In [2]:
def load_df_from_gsheet (wb, table_name):
  df = pd.DataFrame()
  try:
    tbl = wb.worksheet(table_name)
    data = tbl.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
  except:
      print ("Table inexistante !")
  return df

# Sélection d'audios selon critères

In [3]:
saynete_id = "rec7lQw6DtIJP45dn"
language = "rom"
base_dir = "/content/drive/MyDrive/langdeck/assets/audio/voix/"
%cd -q $base_dir
if not os.path.exists(saynete_id):
    print(f"{saynete_id}not exist")
    os.makedirs(saynete_id)
%cd -q $saynete_id

if not os.path.exists(language):
    print(f"{language}not exist")
    os.makedirs(language)
%cd -q $language
!ls

rec6iFAyDnWbY30KJ-rom.mp3  recBcty8NTxqDTVLA-rom.mp3  recPiZ0PPxsx3o1a7-rom.mp3
rec6joo0tfQRmIxRL-rom.mp3  recCQGjqF7GSxwniz-rom.mp3  recQ4laan2j2i1Nj1-rom.mp3
rec7lQw6DtIJP45dn.csv	   recDkf8LsIS4i0Gub-rom.mp3  recr9CS9mbC2WVdHm-rom.mp3
rec7lQw6DtIJP45dn-rom.csv  recItoPa8L282saqj-rom.mp3  recRB9BNBRTq6HWxX-rom.mp3
rec7uqBA4D8chNwQC-rom.mp3  recJ0yx1IhILO3zFD-rom.mp3  recXg83ds1lxtoLjK-rom.mp3
rec8g4gRsRejRuBCw-rom.mp3  recjChTVJzZ1UQUJy-rom.mp3  recyKhQuzHGCh3irl-rom.mp3
recaMm5sdSHaTQo3h-rom.mp3  recLMXR6RDAgUN0u4-rom.mp3  recZFb4HrRekiQsYs-rom.mp3
recBB05hdIHkmPpSp-rom.mp3  recM4NKCmHFimvkkx-rom.mp3
recBbiJPWmVqAv0vq-rom.mp3  recNoby7tUZNH0Hne-rom.mp3


In [11]:

db_lgdk_intgr = dotenv_config["DB_LGDK_INTGR"]
#--- integration database
sheet = db_lgdk_intgr
wb_intgr  = gc.open_by_url(sheet)

df_tbl_phrases = load_df_from_gsheet (wb_intgr, "tbl_phrases")
df_tbl_phrases['phrase_related_story_rec_id'] = df_tbl_phrases['phrase_related_story_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
df_tbl_phrases['phrase_related_story_rec_id'] = df_tbl_phrases['phrase_related_story_rec_id'].map(lambda x:''.join (x))
df_tbl_phrases["phrase_position"] = df_tbl_phrases["phrase_position"].str.zfill(2)
df_tbl_phrases['phrase_words_rec_id'] = df_tbl_phrases['phrase_words_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
#df_tbl_phrases['phrase_words_rec_id'] = df_tbl_phrases['phrase_words_rec_id'].map(lambda x:''.join (x))


df_tbl_phrases = df_tbl_phrases.loc[
    (df_tbl_phrases["phrase_language"]==language)
     & (df_tbl_phrases["phrase_related_story_rec_id"]==saynete_id)
     ][["phrase_translation_id",
        "phrase_audio_url",
        "phrase_position",
        "phrase",
        "phrase_translation",
        "phrase_words_rec_id",
        ]].sort_values(by=['phrase_position'])
#

# récupération des dialogues

In [ ]:
# français
df_tbl_phrases[[
        "phrase_rec_id",
        "phrase_position",
        "phrase",
        ]].to_csv(saynete_id+'.csv', index=False)

In [ ]:
# traduit
df_tbl_phrases[[
        "phrase_translation_id",
        "phrase_position",
        "phrase_translation",
        ]].to_csv(saynete_id+'-'+language+'.csv', index=False)

# récupération des audios


In [ ]:
df_tbl_phrases[[
        "phrase_translation_id",
        "phrase_audio_url",
        ]].apply(
         lambda x:wget.download(
             x.iloc[1],
             x.iloc[0] + ".mp3"),
         axis=1)

,0
273,recRB9BNBRTq6HWxX-rom.mp3
274,recZFb4HrRekiQsYs-rom.mp3
275,recQ4laan2j2i1Nj1-rom.mp3
276,recItoPa8L282saqj-rom.mp3
277,rec6joo0tfQRmIxRL-rom.mp3
278,recBcty8NTxqDTVLA-rom.mp3
279,recCQGjqF7GSxwniz-rom.mp3
280,rec6iFAyDnWbY30KJ-rom.mp3
281,recBB05hdIHkmPpSp-rom.mp3
282,rec7uqBA4D8chNwQC-rom.mp3


# Récupération des mots-clés

In [34]:
if not os.path.exists("keywords"):
    print("keywords not exist")
    os.makedirs("keywords")
%cd -q keywords
!ls

keywords not exist


In [35]:
df_tbl_words = load_df_from_gsheet (wb_intgr, "tbl_words")

In [36]:
vk_word_keys = list(df_tbl_phrases["phrase_words_rec_id"].explode())

In [37]:
df_tbl_words = df_tbl_words.loc[(df_tbl_words["word_language"]==language)
     & (df_tbl_words["word_rec_id"].isin(vk_word_keys))
     ][[
         "word_rec_id",
         "word",
         "word_translation_id",
         "word_translation",
         "word_audio_url",
         "word_notes",
         "word_notes_translation",
     ]]

# Récupération du catalogue des mots-clés

In [38]:
# français
df_tbl_words[[
        "word_rec_id",
        "word",
        "word_notes",
        ]].to_csv(saynete_id+'-_keywords.csv', index=False)

In [39]:
# traduit
df_tbl_words[[
        "word_translation_id",
        "word_translation",
        "word_notes_translation",
        ]].to_csv(saynete_id+'-'+language+'-_keywords.csv', index=False)

# Récupération des audios

In [40]:
df_tbl_words[[
        "word_translation_id",
        "word_audio_url",
        ]].apply(
         lambda x:wget.download(
             x.iloc[1],
             x.iloc[0] + ".mp3"),
         axis=1)

,0
86,recmhUC9Wr0zn4i0o-rom.mp3
87,recSq54lkSPY5m1co-rom.mp3
90,recFYIwEPxr1ffzB6-rom.mp3
91,recfQfsC7kP8CvKFD-rom.mp3
92,recdqvaTPhYuzEG1r-rom.mp3
94,rec7h485P0KpcTReA-rom.mp3
100,recPVdsyMtJvFOetC-rom.mp3
101,recP4UChKgISC2aJS-rom.mp3
